In [1]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine

def get_data_from_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Define the SQLite database file path
sqlite_file = '/home/user/myapp/test.sqlite'

# Create a SQLAlchemy engine and connect to the SQLite database
engine = create_engine(f'sqlite:///{sqlite_file}')

In [2]:

def process_aggregated_insurence_country(data_directory, engine):
    quarter_data = []

    for current_dir, subdirs, files in os.walk(data_directory):
        for file in files:
            file_path = os.path.join(current_dir, file)
            if os.path.isfile(file_path) and file.endswith('.json'):
                parsed_data = get_data_from_file(file_path)

                # Extract the necessary information
                transaction_data = parsed_data['data']['transactionData']

                # Check if there is any data before creating a DataFrame
                if transaction_data:
                    table = []
                    # Extract the necessary information
                    transaction_data = parsed_data['data']['transactionData']

                    # Print the extracted information without headers
                    for transaction in transaction_data:
                        row = {
                            'year': int(os.path.basename(os.path.dirname(file_path))),
                            'quarter': int(file.split('.')[0]),
                            'name': transaction['name'],
                            'count': transaction['paymentInstruments'][0]['count'],
                            'amount': transaction['paymentInstruments'][0]['amount'],
                        }
                        table.append(row)

                    df = pd.DataFrame(table)
                    quarter_data.append(df)

    # Concatenate all DataFrames
    df = pd.concat(quarter_data)

    # Save the DataFrame to the SQLite database table named 'insurance_country'
    df.to_sql('aggregated_insurence_country', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM aggregated_insurence_country;'
    result_df = pd.read_sql(query, engine)
    return result_df
data_directory = '/home/user/pulse/data/aggregated/insurance/country/india'
process_aggregated_insurence_country(data_directory, engine)

,year,quarter,name,count,amount
0,2023,2,Insurance,893850,1.347400e+09
1,2023,1,Insurance,923173,1.408801e+09
2,2023,3,Insurance,1010211,1.445235e+09
3,2023,2,Insurance,1704,2.501107e+06
4,2023,1,Insurance,1520,2.415574e+06
...,...,...,...,...,...
511,2022,3,Insurance,806491,1.054718e+09
512,2022,4,Insurance,1035364,1.439763e+09
513,2020,2,Insurance,185348,3.373217e+07
514,2020,3,Insurance,354284,8.949508e+07


In [3]:
def process_aggregated_insurence_state(data_directory, engine):
    quarter_data = []

    for state in os.listdir(data_directory):
        state_dir = os.path.join(data_directory, state)
        
        for current_dir, subdirs, files in os.walk(state_dir):
            for file in files:
                file_path = os.path.join(current_dir, file)
                if os.path.isfile(file_path) and file.endswith('.json'):
                    parsed_data = get_data_from_file(file_path)

                    table = []
                    # Extract the necessary information
                    from_timestamp = parsed_data['data']['from']
                    to_timestamp = parsed_data['data']['to']
                    transaction_data = parsed_data['data']['transactionData']

                    # Save the extracted information 
                    for transaction in transaction_data:
                        row = {
                            'state': state.lower().replace("&", "-and").replace(" ", "-"),
                            'year': int(os.path.basename(os.path.dirname(file_path))),
                            'quarter': int(file.split('.')[0]),
                            'type_of_transaction': transaction['name'],
                            'number_of_transactions': transaction['paymentInstruments'][0]['count'],
                            'total_amount': transaction['paymentInstruments'][0]['amount'],
                        }
                        table.append(row)

                    df = pd.DataFrame(table)
                    quarter_data.append(df)

    # Concatenate all DataFrames
    df = pd.concat(quarter_data)

    # Save the DataFrame to the SQLite database table named 'aggregated_insurence_state'
    df.to_sql('aggregated_insurence_state', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM aggregated_insurence_state;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_directory_state = '/home/user/pulse/data/aggregated/insurance/country/india/state'

result_df = process_aggregated_insurence_state(data_directory_state, engine)
result_df


,state,year,quarter,type_of_transaction,number_of_transactions,total_amount
0,chandigarh,2023,2,Insurance,1704,2501107.0
1,chandigarh,2023,1,Insurance,1520,2415574.0
2,chandigarh,2023,3,Insurance,1883,2660630.0
3,chandigarh,2021,2,Insurance,378,310943.0
4,chandigarh,2021,1,Insurance,338,238576.0
...,...,...,...,...,...,...
497,himachal-pradesh,2022,3,Insurance,3641,6161692.0
498,himachal-pradesh,2022,4,Insurance,4565,9060436.0
499,himachal-pradesh,2020,2,Insurance,392,77936.0
500,himachal-pradesh,2020,3,Insurance,531,165559.0


In [53]:
import os

data_directory = '/home/user/pulse/data'

for current_dir, subdirs, files in os.walk(data_directory):
    # `current_dir` is the current directory being traversed
    print("Current Directory:", current_dir)

    # `subdirs` is a list of subdirectories in the current directory
    print("Subdirectories:", subdirs)

    # `files` is a list of files in the current directory
    print("Files:", files)

    # Now, you can use `subdirs` to access the subdirectories within the current directory
    for subdir in subdirs:
        sub_dir_path = os.path.join(current_dir, subdir)
        print("Subdirectory Path:", sub_dir_path)

    # Similarly, you can use `files` to access the files within the current directory
    for file in files:
        file_path = os.path.join(current_dir, file)
        print("File Path:", file_path)

    # You can perform additional operations or logic here based on the current directory, subdirectories, and files
    # ...

    print("-" * 50)  # Just for better visualization of different iterations


Current Directory: /home/user/pulse/data
Subdirectories: ['top', 'map', 'aggregated']
Files: []
Subdirectory Path: /home/user/pulse/data/top
Subdirectory Path: /home/user/pulse/data/map
Subdirectory Path: /home/user/pulse/data/aggregated
--------------------------------------------------
Current Directory: /home/user/pulse/data/top
Subdirectories: ['insurance', 'user', 'transaction']
Files: []
Subdirectory Path: /home/user/pulse/data/top/insurance
Subdirectory Path: /home/user/pulse/data/top/user
Subdirectory Path: /home/user/pulse/data/top/transaction
--------------------------------------------------
Current Directory: /home/user/pulse/data/top/insurance
Subdirectories: ['country']
Files: []
Subdirectory Path: /home/user/pulse/data/top/insurance/country
--------------------------------------------------
Current Directory: /home/user/pulse/data/top/insurance/country
Subdirectories: ['india']
Files: []
Subdirectory Path: /home/user/pulse/data/top/insurance/country/india
---------------

In [49]:
data_directory = '/home/user/pulse/data'
for current_dir, subdirs, files in os.walk(data_directory):
    for subdir in os.walk(subdirs):
        if top in subdir:
            pass

        if map in subdir:
            pass

        if aggregated in subdir:
            
            for file in files:
                file_path = os.path.join(current_dir, file)
                if os.path.isfile(file_path) and file.endswith('.json'):
                    if 'insurance' and not 'state' in subdirs:
                        
                            parsed_data = get_data_from_file(file_path)

                            # Extract the necessary information
                            transaction_data = parsed_data['data']['transactionData']

                            # Check if there is any data before creating a DataFrame
                            if transaction_data:
                                table = []
                                # Extract the necessary information
                                transaction_data = parsed_data['data']['transactionData']

                                # Print the extracted information without headers
                                for transaction in transaction_data:
                                    row = {
                                        'year': int(os.path.basename(os.path.dirname(file_path))),
                                        'quarter': int(file.split('.')[0]),
                                        'name': transaction['name'],
                                        'count': transaction['paymentInstruments'][0]['count'],
                                        'amount': transaction['paymentInstruments'][0]['amount'],
                                    }
                                    table.append(row)

                        
                                df = pd.DataFrame(table)
                                quarter_data.append(df)
                    if 'insurance' and 'state' in subdirs:
                            parsed_data = get_data_from_file(file_path)

                            table = []
                            # Extract the necessary information
                            from_timestamp = parsed_data['data']['from']
                            to_timestamp = parsed_data['data']['to']
                            transaction_data = parsed_data['data']['transactionData']

                            # Save the extracted information 
                            for transaction in transaction_data:
                                row = {
                                    'state': state.lower().replace("&", "-and").replace(" ", "-"),
                                    'year': int(os.path.basename(os.path.dirname(file_path))),
                                    'quarter': int(file.split('.')[0]),
                                    'from_timestamp': from_timestamp,
                                    'to_timestamp': to_timestamp,
                                    'type_of_transaction': transaction['name'],
                                    'number_of_transactions': transaction['paymentInstruments'][0]['count'],
                                    'total_amount': transaction['paymentInstruments'][0]['amount'],
                                }
                                table.append(row)

                    df = pd.DataFrame(table)
                    quarter_data.append(df)

                








    # Concatenate all DataFrames
    df = pd.concat(quarter_data)

    # Save the DataFrame to the SQLite database table named 'insurance_country'
    df.to_sql(f'{aggregated}_{insurence}_{country}', engine, index=False, if_exists='replace')

      

top
map
aggregated
insurance
user
transaction
country
india
2023
state
2021
2022
2020
chandigarh
sikkim
karnataka
jharkhand
chhattisgarh
maharashtra
andhra-pradesh
meghalaya
dadra-&-nagar-haveli-&-daman-&-diu
west-bengal
assam
uttar-pradesh
odisha
rajasthan
madhya-pradesh
tamil-nadu
ladakh
lakshadweep
punjab
andaman-&-nicobar-islands
kerala
uttarakhand
nagaland
telangana
delhi
goa
mizoram
gujarat
jammu-&-kashmir
arunachal-pradesh
puducherry
haryana
bihar
manipur
tripura
himachal-pradesh
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
2021
2022
2020
2023
202

In [16]:
import os
import pandas as pd

def process_aggregated_transaction_country(data_directory, engine):
    quarter_data = []

    for current_dir, subdirs, files in os.walk(data_directory):
        if 'state' in subdirs:
            break
        else:
            for file in files:
                file_path = os.path.join(current_dir, file)
                if os.path.isfile(file_path) and file.endswith('.json'):
                    parsed_data = get_data_from_file(file_path)

                    table = []
                    # Extract the necessary information
                    transaction_data = parsed_data['data']['transactionData']

                    # Save the extracted information without headers
                    for transaction in transaction_data:
                        row = {
                            'year': int(os.path.basename(os.path.dirname(file_path))),
                            'quarter': int(file.split('.')[0]),
                            'name': transaction['name'],
                            'count': transaction['paymentInstruments'][0]['count'],
                            'amount': transaction['paymentInstruments'][0]['amount'],
                        }
                        table.append(row)

                    df = pd.DataFrame(table)
                    quarter_data.append(df)

    # Check if there are any DataFrames to concatenate
    if quarter_data:
        # Concatenate all DataFrames
        df = pd.concat(quarter_data)

        # Save the DataFrame to the SQLite database table named 'aggregated_transaction_country'
        df.to_sql('aggregated_transaction_country', engine, index=False, if_exists='replace')
    else:
        # print("No eligible files found for concatenation.")
        pass
    # Query the SQLite database to verify the data
    query = 'SELECT * FROM aggregated_transaction_country;'
    result_df = pd.read_sql(query, engine)
    return result_df

data_aggregated_transaction_country = '/home/user/pulse/data/aggregated/transaction/country/india/'
result_df = process_aggregated_transaction_country(data_aggregated_transaction_country, engine)
result_df


,year,quarter,name,count,amount
0,2019,2,Peer-to-peer payments,384711008,1.196425e+12
1,2019,2,Recharge & bill payments,209898072,7.686585e+10
2,2019,2,Merchant payments,206467182,7.341584e+10
3,2019,2,Financial Services,2650384,5.453293e+08
4,2019,2,Others,11656557,6.969424e+09
...,...,...,...,...,...
4244,2020,4,Peer-to-peer payments,1249512647,4.474548e+12
4245,2020,4,Merchant payments,1127490136,6.848736e+11
4246,2020,4,Recharge & bill payments,479784084,2.278758e+11
4247,2020,4,Financial Services,2942567,3.259168e+09


In [15]:
def process_aggregated_transaction_state(data_directory, engine):
    quarter_data = []

    for state in os.listdir(data_directory):
        state_dir = os.path.join(data_directory, state)
        
        for current_dir, subdirs, files in os.walk(state_dir):
            for file in files:
                file_path = os.path.join(current_dir, file)
                if os.path.isfile(file_path) and file.endswith('.json'):
                    parsed_data = get_data_from_file(file_path)

                    table = []
                    # Extract the necessary information
                    from_timestamp = parsed_data['data']['from']
                    to_timestamp = parsed_data['data']['to']
                    transaction_data = parsed_data['data']['transactionData']

                    # Save the data
                    for transaction in transaction_data:
                        row = {
                            'state': state.lower().replace("&", "-and").replace(" ", "-"),
                            'year': int(os.path.basename(os.path.dirname(file_path))),
                            'quarter': int(file.split('.')[0]),
                            # 'from_timestamp': from_timestamp,
                            # 'to_timestamp': to_timestamp,
                            'type_of_transaction': transaction['name'],
                            'number_of_transactions': transaction['paymentInstruments'][0]['count'],
                            'total_amount': transaction['paymentInstruments'][0]['amount'],
                        }
                        table.append(row)

                    df = pd.DataFrame(table)
                    quarter_data.append(df)

    # Concatenate all DataFrames
    df = pd.concat(quarter_data)

    # Save the DataFrame to the SQLite database table named 'aggregated_transaction_state'
    df.to_sql('aggregated_transaction_state', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM aggregated_transaction_state;'
    result_df = pd.read_sql(query, engine)
    return result_df
data_aggregated_transaction_state = '/home/user/pulse/data/aggregated/transaction/country/india/state/'
result_df = process_aggregated_transaction_state(data_aggregated_transaction_state, engine)
result_df


,state,year,quarter,type_of_transaction,number_of_transactions,total_amount
0,chandigarh,2019,2,Peer-to-peer payments,604849,2.339922e+09
1,chandigarh,2019,2,Recharge & bill payments,342294,1.856590e+08
2,chandigarh,2019,2,Merchant payments,277811,1.527669e+08
3,chandigarh,2019,2,Financial Services,5590,8.397652e+05
4,chandigarh,2019,2,Others,21926,1.392805e+07
...,...,...,...,...,...,...
4129,himachal-pradesh,2020,4,Recharge & bill payments,2638030,9.363058e+08
4130,himachal-pradesh,2020,4,Peer-to-peer payments,2594659,1.142627e+10
4131,himachal-pradesh,2020,4,Merchant payments,2001259,1.589433e+09
4132,himachal-pradesh,2020,4,Financial Services,11353,1.553425e+07


In [16]:
def process_aggregated_user_country(data_directory, engine):
    quarter_data = []

    for current_dir, subdirs, files in os.walk(data_directory):
        for file in files:
            file_path = os.path.join(current_dir, file)
            if os.path.isfile(file_path) and file.endswith('.json'):
                parsed_data = get_data_from_file(file_path)

                table = []
                # Extract the necessary information
                registered_users = parsed_data["data"]["aggregated"]["registeredUsers"]
                total_open_apps = parsed_data["data"]["aggregated"]["appOpens"]
                users_by_device = parsed_data["data"]["usersByDevice"]

                # Check if users_by_device is not None before iterating
                if users_by_device is not None:
                    # Save the extracted information
                    for user_device in users_by_device:
                        row = {
                            'year': int(os.path.basename(os.path.dirname(file_path))),
                            'quarter': int(file.split('.')[0]),
                            "registered_users": registered_users,
                            "total_open_apps": total_open_apps,
                            "phone_brand": user_device["brand"],
                            "phone_count": user_device["count"],
                            "Percentage": f"{user_device['percentage'] * 100:.2f}",
                        }
                        table.append(row)

                df = pd.DataFrame(table)
                quarter_data.append(df)

    # Concatenate all DataFrames
    df = pd.concat(quarter_data)


    # Save the DataFrame to the SQLite database table named 'aggregated_user_country'
    df.to_sql('aggregated_user_country', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM aggregated_user_country;'
    result_df = pd.read_sql(query, engine)
    return result_df


# Example usage:
data_aggregated_user_country = '/home/user/pulse/data/aggregated/user/country/india/'
result_df = process_aggregated_user_country(data_aggregated_user_country, engine)
result_df


,year,quarter,registered_users,total_open_apps,phone_brand,phone_count,Percentage
0,2019,2,141808226,920413791,Xiaomi,36936092,26.05
1,2019,2,141808226,920413791,Samsung,28857066,20.35
2,2019,2,141808226,920413791,Vivo,22157338,15.62
3,2019,2,141808226,920413791,Oppo,15797055,11.14
4,2019,2,141808226,920413791,Realme,6829373,4.82
...,...,...,...,...,...,...,...
6914,2020,4,264271293,7461623162,Motorola,5650618,2.14
6915,2020,4,264271293,7461623162,Huawei,4915828,1.86
6916,2020,4,264271293,7461623162,OnePlus,4621214,1.75
6917,2020,4,264271293,7461623162,Lenovo,3880170,1.47


In [17]:
def process_aggregated_user_state(data_directory, engine):
    quarter_data = []

    for state in os.listdir(data_directory):
        state_dir = os.path.join(data_directory, state)
        
        for current_dir, subdirs, files in os.walk(state_dir):
            for file in files:
                file_path = os.path.join(current_dir, file)
                if os.path.isfile(file_path) and file.endswith('.json'):
                    parsed_data = get_data_from_file(file_path)

                    # Extract the necessary information
                    registered_users = parsed_data["data"]["aggregated"]["registeredUsers"]
                    total_open_apps = parsed_data["data"]["aggregated"]["appOpens"]
                    users_by_device = parsed_data["data"]["usersByDevice"]

                    # Check if users_by_device is not None before iterating
                    if users_by_device is not None:
                        # Save the extracted information
                        for user_device in users_by_device:
                            row = {
                                'state': state.lower().replace("&", "-and").replace(" ", "-"),
                                'year': int(os.path.basename(os.path.dirname(file_path))),
                                'quarter': int(file.split('.')[0]),
                                "registered_users": registered_users,
                                "total_open_apps": total_open_apps,
                                "phone_brand": user_device["brand"],
                                "phone_count": user_device["count"],
                                "Percentage": f"{user_device['percentage'] * 100:.2f}",
                            }
                            quarter_data.append(row)

    # Concatenate all DataFrames
    df = pd.DataFrame(quarter_data)

    # Save the DataFrame to the SQLite database table named 'aggregated_user_state'
    df.to_sql('aggregated_user_state', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM aggregated_user_state;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_directory_user_state = '/home/user/pulse/data/aggregated/user/country/india/state'
result_df_user_state = process_aggregated_user_state(data_directory_user_state, engine)
result_df_user_state

,state,year,quarter,registered_users,total_open_apps,phone_brand,phone_count,Percentage
0,chandigarh,2019,2,240439,1220176,Xiaomi,58356,24.27
1,chandigarh,2019,2,240439,1220176,Samsung,48067,19.99
2,chandigarh,2019,2,240439,1220176,Vivo,34791,14.47
3,chandigarh,2019,2,240439,1220176,Oppo,27444,11.41
4,chandigarh,2019,2,240439,1220176,Apple,20041,8.34
...,...,...,...,...,...,...,...,...
6727,himachal-pradesh,2020,4,1508766,22001148,Motorola,29156,1.93
6728,himachal-pradesh,2020,4,1508766,22001148,Lava,29138,1.93
6729,himachal-pradesh,2020,4,1508766,22001148,Gionee,23461,1.55
6730,himachal-pradesh,2020,4,1508766,22001148,Micromax,23015,1.53


In [18]:

def process_map_insurance_hover_country(data_directory, engine):
    quarter_data = []

    for current_dir, subdirs, files in os.walk(data_directory):
        for file in files:
            file_path = os.path.join(current_dir, file)
            if os.path.isfile(file_path) and file.endswith('.json'):
                parsed_data = get_data_from_file(file_path)

                table = []
                # Extract the necessary information
                hover_data_list = parsed_data["data"]["hoverDataList"]

                # Save the extracted information
                for entry in hover_data_list:
                    metric_data = entry["metric"][0]  # Extracting the first element of the metric array
                    row = {
                        'year': int(os.path.basename(os.path.dirname(file_path))),
                        'quarter': int(file.split('.')[0]),
                        'state_name': entry["name"],
                        'total_transactions_count': metric_data["count"],
                        'total_transactions_amount': metric_data["amount"],
                    }
                    table.append(row)

                df = pd.DataFrame(table)
                quarter_data.append(df)

    # Concatenate all DataFrames
    df = pd.concat(quarter_data)

    # Save the DataFrame to the SQLite database table named 'map_insurance_hover_country'
    df.to_sql('map_insurance_hover_country', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM map_insurance_hover_country;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_map_insurance_hover_country = '/home/user/pulse/data/map/insurance/hover/country/india'
process_map_insurance_hover_country(data_map_insurance_hover_country, engine)


,year,quarter,state_name,total_transactions_count,total_transactions_amount
0,2023,2,puducherry,1809,2614006.0
1,2023,2,tamil nadu,79698,105328191.0
2,2023,2,uttar pradesh,74194,122522457.0
3,2023,2,madhya pradesh,27027,44871764.0
4,2023,2,andhra pradesh,36376,52067478.0
...,...,...,...,...,...
10598,2020,4,jammu & kashmir,647,734861.0
10599,2020,4,goa,614,682364.0
10600,2020,4,arunachal pradesh,125,84247.0
10601,2020,4,delhi,9725,8045442.0


In [20]:

def process_map_transaction_hover_state(data_directory, engine):
    quarter_data = []

    for state in os.listdir(data_directory):
        state_dir = os.path.join(data_directory, state)
        
        for current_dir, subdirs, files in os.walk(state_dir):
            for file in files:
                file_path = os.path.join(current_dir, file)
                if os.path.isfile(file_path) and file.endswith('.json'):
                    parsed_data = get_data_from_file(file_path)

                    table = []
                    # Extract the necessary information
                    hover_data_list = parsed_data["data"]["hoverDataList"]

                    # Save the extracted information
                    for entry in hover_data_list:
                        district_name = entry["name"]
                        metric_data = entry["metric"][0]
                        total_transactions_count = metric_data["count"]
                        total_transactions_amount = metric_data["amount"]

                        row = {
                            'year': int(os.path.basename(os.path.dirname(file_path))),
                            'quarter': int(file.split('.')[0]),
                            'state_name/ut_name': state,
                            'districts_name': district_name,
                            'total_transactions_count': total_transactions_count,
                            'total_transactions_amount': total_transactions_amount,
                        }
                        table.append(row)

                    df = pd.DataFrame(table)
                    quarter_data.append(df)

    # Concatenate all DataFrames
    df = pd.concat(quarter_data)

    # Save the DataFrame to the SQLite database table named 'map_transaction_hover_state'
    df.to_sql('map_transaction_hover_state', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM map_transaction_hover_state;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_map_transaction_hover_state = '/home/user/pulse/data/map/transaction/hover/country/india/state'
process_map_transaction_hover_state(data_map_transaction_hover_state, engine)


,year,quarter,state_name/ut_name,districts_name,total_transactions_count,total_transactions_amount
0,2019,2,chandigarh,chandigarh district,1252470,2.693116e+09
1,2019,1,chandigarh,chandigarh district,1274476,2.396032e+09
2,2019,3,chandigarh,chandigarh district,1506543,3.269682e+09
3,2019,4,chandigarh,chandigarh district,1850915,4.152514e+09
4,2023,2,chandigarh,chandigarh district,11553687,1.441441e+10
...,...,...,...,...,...,...
16827,2020,4,himachal-pradesh,kullu district,468786,9.640804e+08
16828,2020,4,himachal-pradesh,mandi district,671714,1.247938e+09
16829,2020,4,himachal-pradesh,lahul and spiti district,29377,5.244902e+07
16830,2020,4,himachal-pradesh,chamba district,304032,5.078800e+08


In [21]:


def process_map_user_hover_country(data_directory, engine):
    quarter_data = []

    for current_dir, subdirs, files in os.walk(data_directory):
        for file in files:
            file_path = os.path.join(current_dir, file)
            if os.path.isfile(file_path) and file.endswith('.json'):
                parsed_data = get_data_from_file(file_path)


                table = []
                # Extract the necessary information
                hover_data = parsed_data["data"]["hoverData"]

                # Extracted information for each state/UT
                for state, state_data in hover_data.items():
                    row = {
                        'year': int(os.path.basename(os.path.dirname(file_path))),
                        'quarter': int(file.split('.')[0]),
                        'state/ut_name': state,
                        'registered_users': state_data["registeredUsers"],
                    }
                    table.append(row)

                df = pd.DataFrame(table)
                quarter_data.append(df)

    # Concatenate all DataFrames
    df = pd.concat(quarter_data)

    # Save the DataFrame to the SQLite database table named 'map_user_hover_country'
    df.to_sql('map_user_hover_country', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM map_user_hover_country;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_directory_user_hover_country = '/home/user/pulse/data/map/user/hover/country/india'
process_map_user_hover_country(data_directory_user_hover_country, engine)


,year,quarter,state/ut_name,registered_users
0,2019,2,puducherry,156458
1,2019,2,tamil nadu,7429609
2,2019,2,uttar pradesh,14316349
3,2019,2,madhya pradesh,7301172
4,2019,2,andhra pradesh,9389003
...,...,...,...,...
17659,2020,4,jammu & kashmir,1079022
17660,2020,4,goa,527584
17661,2020,4,arunachal pradesh,231292
17662,2020,4,delhi,9747968


In [3]:

def process_top_insurance_country(data_directory):
    quarter_data = []

    for root, dirs, files in os.walk(data_directory):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")
                parsed_data = get_data_from_file(file_path)

                for entity_type in ['states', 'districts', 'pincodes']:
                    entity_data = [] 
                    entities = parsed_data.get('data', {}).get(entity_type, [])  # Use get() to handle missing keys

                    for entity in entities:
                        entity_row = {
                            'year': int(os.path.basename(os.path.dirname(file_path))),
                            'quarter': int(file.split('.')[0]),
                            'entity_type': entity_type,
                            'entity_name': entity.get('entityName', ''),
                            'transaction_type': entity.get('metric', {}).get('type', ''),
                            'count': entity.get('metric', {}).get('count', 0),
                            'amount': entity.get('metric', {}).get('amount', 0),
                        }

                        entity_data.append(pd.DataFrame([entity_row]))

                    quarter_data.extend(entity_data)

    df = pd.concat(quarter_data, ignore_index=True)
    # df.to_csv('top_insurence_country.csv', index=False)
    # Save the DataFrame to the SQLite database table named 'map_user_hover_country'
    df.to_sql('top_insurence_country', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM top_insurence_country;'
    result_df = pd.read_sql(query, engine)
    return result_df

    return df

# Example usage:
data_directory = '/home/user/pulse/data/top/insurance/country/india'
result_df = process_top_insurance_country(data_directory)
print(result_df)


Processing file: /home/user/pulse/data/top/insurance/country/india/2023/2.json
Processing file: /home/user/pulse/data/top/insurance/country/india/2023/1.json
Processing file: /home/user/pulse/data/top/insurance/country/india/2023/3.json
Processing file: /home/user/pulse/data/top/insurance/country/india/state/chandigarh/2023/2.json


TypeError: 'NoneType' object is not iterable

In [5]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine

def get_data_from_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def process_top_insurance_state(data_directory, engine):
    quarter_data = []

    for state in os.listdir(data_directory):
        state_dir = os.path.join(data_directory, state)
        for year in range(2018, 2023):
            for quarter in range(1, 5):
                file_path = os.path.join(state_dir, f'{year}/{quarter}.json')
                if os.path.exists(file_path):
                    parsed_data = get_data_from_file(file_path)

                    for entity_type in ['states', 'districts', 'pincodes']:
                        entities = parsed_data['data'].get(entity_type)

                        if entities is not None:
                            entity_data = []

                            for entity in entities:
                                entity_name = entity["entityName"]
                                entity_metric = entity["metric"]

                                row = {
                                    'year': year,
                                    'quarter': quarter,
                                    'entity_type': entity_type,
                                    'entity_name': entity_name,
                                    'transaction_type': entity_metric['type'],
                                    'count': entity_metric['count'],
                                    'amount': entity_metric['amount'],
                                }
                                entity_data.append(row)

                            entity_df = pd.DataFrame(entity_data)
                            quarter_data.append(entity_df)

    df = pd.concat(quarter_data)
    df.to_sql('top_insurance_state', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM top_insurance_state;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_directory = '/home/user/pulse/data/top/insurance/country/india/state'

result_df = process_top_insurance_state(data_directory, engine)
print(result_df)


      year  quarter entity_type entity_name transaction_type  count    amount
0     2020        2   districts  chandigarh            TOTAL    149   29537.0
1     2020        2    pincodes      160101            TOTAL     31    6158.0
2     2020        2    pincodes      160047            TOTAL     25    4718.0
3     2020        2    pincodes      160036            TOTAL     16    3261.0
4     2020        2    pincodes      160023            TOTAL     12    2002.0
...    ...      ...         ...         ...              ...    ...       ...
7065  2022        4    pincodes      173220            TOTAL     67  128773.0
7066  2022        4    pincodes      173212            TOTAL     67  117038.0
7067  2022        4    pincodes      176001            TOTAL     65  113046.0
7068  2022        4    pincodes      176061            TOTAL     65  146613.0
7069  2022        4    pincodes      173001            TOTAL     61  101537.0

[7070 rows x 7 columns]


In [6]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine

def get_data_from_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def process_top_transaction_country(data_directory, engine):
    quarter_data = []

    for year in range(2018, 2023):
        for quarter in range(1, 5):
            file_path = os.path.join(data_directory, f'{year}/{quarter}.json')
            if os.path.exists(file_path):
                parsed_data = get_data_from_file(file_path)

                for entity_type in ['states', 'districts', 'pincodes']:
                    entity_data = []  

                    entities = parsed_data['data'][entity_type]

                    for entity in entities:
                        entity_row = {
                            'year': year,
                            'quarter': quarter,
                            'entity_type': entity_type,
                            'entity_name': entity['entityName'],
                            'transaction_type': entity['metric']['type'],  
                            'count': entity['metric']['count'],
                            'amount': entity['metric']['amount'],
                        }
                    
                        entity_data.append(entity_row)

                    entity_df = pd.DataFrame(entity_data) 
                    quarter_data.append(entity_df)  

    df = pd.concat(quarter_data)
    df.to_sql('top_transaction_country', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM top_transaction_country;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_directory = '/home/user/pulse/data/top/transaction/country/india'

result_df = process_top_transaction_country(data_directory, engine)
print(result_df)


     year  quarter entity_type     entity_name transaction_type      count  \
0    2018        1      states     maharashtra            TOTAL   16387034   
1    2018        1      states   uttar pradesh            TOTAL   12537805   
2    2018        1      states       karnataka            TOTAL   12016899   
3    2018        1      states     west bengal            TOTAL   11710225   
4    2018        1      states  andhra pradesh            TOTAL    9039585   
..    ...      ...         ...             ...              ...        ...   
595  2022        4    pincodes          480001            TOTAL  240053791   
596  2022        4    pincodes          400070            TOTAL  180508440   
597  2022        4    pincodes          800001            TOTAL  167166031   
598  2022        4    pincodes          751001            TOTAL  140210762   
599  2022        4    pincodes          700039            TOTAL  119193886   

           amount  
0    2.171161e+10  
1    1.393997e+10  
2  

In [28]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine

def get_data_from_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def process_top_transaction_state(data_directory, engine):
    quarter_data = []

    for state in os.listdir(data_directory):
        state_dir = os.path.join(data_directory, state)
        for year in range(2018, 2023):
            for quarter in range(1, 5):
                file_path = os.path.join(state_dir, f'{year}/{quarter}.json')
                if os.path.exists(file_path):
                    parsed_data = get_data_from_file(file_path)

                    for entity_type in ['states', 'districts', 'pincodes']:
                        entities = parsed_data['data'].get(entity_type)

                        if entities is not None:
                            entity_data = []  

                            for entity in entities:
                                entity_name = entity["entityName"]
                                entity_metric = entity["metric"]

                                row = {
                                    'year': year,
                                    'quarter': quarter,
                                    'entity_type': entity_type,
                                    'entity_name': entity_name,
                                    'transaction_type': entity_metric['type'],
                                    'count': entity_metric['count'],
                                    'amount': entity_metric['amount'],
                                }
                                entity_data.append(row)

                            entity_df = pd.DataFrame(entity_data)
                            quarter_data.append(entity_df)

    df = pd.concat(quarter_data)
    df.to_sql('top_transaction_state', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM top_transaction_state;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_directory = '/home/user/pulse/data/top/transaction/country/india/state'
result_df = process_top_transaction_state(data_directory, engine)
print(result_df)


       year  quarter entity_type entity_name transaction_type   count  \
0      2018        1   districts  chandigarh            TOTAL  381895   
1      2018        1    pincodes      160008            TOTAL  204745   
2      2018        1    pincodes      160101            TOTAL   26317   
3      2018        1    pincodes      160002            TOTAL   23236   
4      2018        1    pincodes      160036            TOTAL   18446   
...     ...      ...         ...         ...              ...     ...   
13054  2022        4    pincodes      175001            TOTAL  609639   
13055  2022        4    pincodes      173025            TOTAL  609183   
13056  2022        4    pincodes      175101            TOTAL  557239   
13057  2022        4    pincodes      173212            TOTAL  425394   
13058  2022        4    pincodes      177001            TOTAL  406943   

             amount  
0      6.134354e+08  
1      3.034820e+08  
2      4.562948e+07  
3      3.400308e+07  
4      3.0891

In [7]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine

def get_data_from_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def process_top_user_country(data_directory, engine):
    quarter_data = []

    for year in range(2018, 2023):
        for quarter in range(1, 5):
            file_path = os.path.join(data_directory, f'{year}/{quarter}.json')
            if os.path.exists(file_path):
                parsed_data = get_data_from_file(file_path)

                for entity_type in ['states', 'districts', 'pincodes']:
                    entities = parsed_data['data'][entity_type]

                    entity_data = []  

                    for entity in entities:
                        entity_row = {
                            'year': year,
                            'quarter': quarter,
                            'entity_type': entity_type,
                            'entity_name': entity['name'],
                            'registeredUsers': entity['registeredUsers'],
                        }
                        entity_data.append(entity_row)

                    entity_df = pd.DataFrame(entity_data)
                    quarter_data.append(entity_df)

    df = pd.concat(quarter_data)
    df.to_sql('top_user_country', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM top_user_country;'
    result_df = pd.read_sql(query, engine)
    return result_df

# Example usage:
data_directory = '/home/user/pulse/data/top/user/country/india'

result_df = process_top_user_country(data_directory, engine)
print(result_df)


     year  quarter entity_type     entity_name  registeredUsers
0    2018        1      states     maharashtra          6106994
1    2018        1      states   uttar pradesh          4694250
2    2018        1      states       karnataka          3717763
3    2018        1      states  andhra pradesh          3336450
4    2018        1      states       telangana          3315560
..    ...      ...         ...             ...              ...
595  2022        4    pincodes          122001           542209
596  2022        4    pincodes          110092           534009
597  2022        4    pincodes          110086           477514
598  2022        4    pincodes          121004           468607
599  2022        4    pincodes          201009           451153

[600 rows x 5 columns]


In [ ]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine
 
# Function to get data from file
def get_data_from_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Function to process aggregated user country data
def process_aggregated_user_country(data_directory, engine):
    quarter_data = []

    for root, dirs, files in os.walk(data_directory):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")
                parsed_data = get_data_from_file(file_path)

                # Check if 'aggregated' key exists in parsed_data['data']
                if 'data' in parsed_data and 'aggregated' in parsed_data['data']:
                    aggregated_data = parsed_data['data']['aggregated']
                    registered_users = aggregated_data.get('registeredUsers')
                    total_open_apps = aggregated_data.get('appOpens')
                    users_by_device = aggregated_data.get('usersByDevice')
 
                    if users_by_device is not None:
                        table = []
                        # Save the extracted information
                        for user_device in users_by_device:
                            row = {
                                'year': int(os.path.basename(os.path.dirname(file_path))),
                                'quarter': int(file.split('.')[0]),
                                'registered_users': registered_users,
                                'total_open_apps': total_open_apps,
                                'phone_brand': user_device['brand'],
                                'phone_count': user_device['count'],
                                'Percentage': f"{user_device['percentage'] * 100:.2f}",
                            }
                            table.append(row)
 
                        df = pd.DataFrame(table)
                        quarter_data.append(df)

    # Concatenate all DataFrames
    result_df = pd.concat(quarter_data, ignore_index=True)

    # Save the DataFrame to the SQLite database
    result_df.to_sql('aggregated_user_country', engine, index=False, if_exists='replace')

    # Query the SQLite database to verify the data
    query = 'SELECT * FROM aggregated_user_country;'
    result_df = pd.read_sql(query, engine)
    return result_df


data_directory = '/home/user/pulse/data/aggregated/user/country/india/'
# Replace 'sqlite:///your_database_name.db' with the actual path and name of your SQLite database
engine = create_engine('sqlite:///phonepe.sqlite')
result_df = process_aggregated_user_country(data_directory, engine)
print(result_df)
